In [2]:
%pip install transformers langdetect deep-translator torch torchvision torchaudio langchain langchain-community langchain-ollama gradio -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 113.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 116.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━

In [3]:
from langchain.prompts import PromptTemplate
from langchain_ollama import ChatOllama

In [4]:
llm = ChatOllama(
    model = "mistral",
    temperature = 0.2,
    max_tokens = 256
)

In [5]:
template = """
You're a native internal AI Assistant, who helps a lot to the user.
Help users with their important tasks, like a professor in a particular field.

Query: {query}
"""

prompt = PromptTemplate(
    input_variables = ["query"],
    template = template,
)

chain = prompt | llm

In [6]:
from langdetect import detect
from deep_translator import GoogleTranslator

class Translation:
  def __init__(self, text, destination):
    self.text = text
    self.destination = destination

    try:
      self.original = detect(self.text)
    except Exception as error:
      print(f"Error Occurred, Details : {error}")
      self.original = "auto"

  def translate(self):
    translator = GoogleTranslator(source=self.original, target=self.destination)
    translation = translator.translate(self.text)

    return translation

In [7]:
def reply(message, history):
  txt = Translation(message, "en")

  if txt.original == "en":
    response = chain.invoke({
        "query": message
    })

    return response
  else:
    translation = txt.translate()

    response = chain.invoke({
        "query": translation
    })

    t = Translation(response.content, txt.original)
    final_response = t.translate()

    return final_response

In [9]:
question = "how do you earn more money through Real Estate Investment?"

result = reply(question, history = [])

result.content

' To increase your earnings through real estate investment, consider the following strategies:\n\n1. Buy and Hold: Purchase properties with potential for long-term appreciation and generate income through rental cash flow.\n\n2. Flipping Properties: Buy low, renovate or rehab, then sell high within a short period of time. This strategy requires knowledge of market trends, construction costs, and the ability to manage projects efficiently.\n\n3. Real Estate Investment Trusts (REITs): These are companies that own, operate, or finance income-producing real estate. By investing in REITs, you can earn dividends from their rental income without directly owning properties.\n\n4. Real Estate Crowdfunding: Pool your money with other investors to invest in larger projects. This allows you to diversify your portfolio and potentially earn higher returns compared to individual property investments.\n\n5. Commercial Real Estate: Invest in commercial properties such as office buildings, retail spaces

In [13]:
question = "आप रियल एस्टेट निवेश के माध्यम से अधिक पैसा कैसे कमा सकते हैं?"

result = reply(question, history = [])

result

'रियल एस्टेट निवेश के ज़रिए अपनी आय बढ़ाने के लिए, निम्नलिखित रणनीतियों पर विचार करें:\n\n1. खरीदें और रखें रणनीति: इसमें लंबे समय तक मूल्य वृद्धि के लिए उन्हें रखने के इरादे से संपत्ति खरीदना शामिल है। आप संपत्ति के मूल्य में वृद्धि की प्रतीक्षा करते हुए किराये के भुगतान से आय उत्पन्न कर सकते हैं।\n\n2. फ़्लिपिंग प्रॉपर्टी: कम कीमत पर खरीदना, संपत्ति का नवीनीकरण या सुधार करना और उसे अधिक कीमत पर बेचना फ़्लिपिंग कहलाता है। इस रणनीति के लिए बाज़ार के रुझान, नवीनीकरण लागत और त्वरित निर्णय लेने के कौशल के बारे में जानकारी की आवश्यकता होती है।\n\n3. रियल एस्टेट निवेश ट्रस्ट (REIT): ये ऐसी कंपनियाँ हैं जो आय-उत्पादक रियल एस्टेट का स्वामित्व रखती हैं, उसका संचालन करती हैं या उसका वित्तपोषण करती हैं। वे व्यक्तिगत निवेशकों को व्यापक पूंजी या संपत्ति प्रबंधन विशेषज्ञता की आवश्यकता के बिना बड़े पैमाने पर रियल एस्टेट परियोजनाओं में निवेश करने की अनुमति देते हैं।\n\n4. हाउस हैकिंग: इस रणनीति में एक मल्टीफ़ैमिली प्रॉपर्टी खरीदना और एक यूनिट में रहना और अपने बंधक भुगतान को कवर करने के लिए अन्य को कि